# Segmentation using the qim.register module
*TODO: Figure our how to "install" the module so that I don't need to put the github folder in the path.*

## Load modules

In [1]:
import sys, os
#sys.path.insert(0, '/home/hektor/scripts')
sys.path.insert(0, '/lunarc/nobackup/projects/qim')

import qim.registration as registration
import qim.tools as tools
import tifffile
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

from skimage import filters


## Read data
The data is from Elins rat bone

In [2]:
qimfolder = '/lunarc/nobackup/projects/qim/QIM_2020_LU_Elin_Bio'
datapath = os.path.join(qimfolder,'pvt21/reg-common-peaks_new')
x_data = os.path.join(datapath,'x_registered.tiff')
n_data = os.path.join(datapath,'n-bin1.tif')

x_volume = tifffile.imread(x_data)
n_volume = tifffile.imread(n_data)


## Create the dual histogram

The x-ray data will be on the x-axis and the neutron data on the y-axis. The number of bins will be the same on both axes.

*TODO: perhaps it is useful to have different number of bins on the axes*


In [3]:
nbins = 256
H,xedges,yedges = registration.make_dual_histogram(x_volume,n_volume,nbins)

#H0 = H[0,0]
xc = tools.find_bin_centers(xedges)
yc = tools.find_bin_centers(yedges)
for i,x in enumerate(xc):
    for j,y in enumerate(yc):
        if (x < 30000) and (y<30000):
            H[i,j]=0
#H[0,0]=H0
#make a plot
registration.plot_dual_histogram(H,xedges,yedges,xlabel='X-ray intensity',ylabel='Neutron intensity',vmax=1500)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Fit Gaussians to the histogram
This will automatically look for maximas of the histogram.

*TODO: Make it work better for peaks at the edges*


In [12]:
# generate markers
ml = [[0,2500],[8000,0],[65400,8700]]
ml = [[0,2500],[7175, 3139], [20800, 46000], [25800, 15900], [48000, 28000], [66085, 16200]]#x reg onto n
ml = [[20800, 46000], [35000, 30000], [65400, 16200]]#x reg onto n
markers = registration.setup_markers(H,xedges,yedges,markers=ml)
# Fit a Gaussian to each marker
opts = registration.fit_gaussians(H,markers,xedges,yedges,fitdist=20)
# Make a plot with markers and Gaussians
registration.plot_dual_histogram(H,xedges,yedges,xlabel='X-ray intensity',ylabel='Neutron intensity',vmax=1500,markers=markers,fits=opts)

Trying to fit peak at (20863.7, 46207.3)
Initial guess: [20863.682 46207.295 100.0 100.0 0.0 1462.000000 0.0]
Fitted parameters [19855.249 45649.577 3106.0 2460.2 -1.8 1845.996288 -2.0]
Fitted 1 peaks
Trying to fit peak at (35199.5, 30079.5)
Initial guess: [35199.463 30079.541 100.0 100.0 0.0 3271.000000 0.0]
Fitted parameters [36630.630 28768.173 3078.6 2397.3 -0.6 6268.389957 -3.6]
Fitted 2 peaks
Trying to fit peak at (65407.0, 16255.8)
Marker is on an edge
Initial guess: [65407.002 16255.752 100.0 100.0 0.0 112817.000000 0.0]
Fitted parameters [65452.993 15306.796 60.3 1567.5 0.0 198735.402787 5.4]
Fitted 3 peaks


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Build phase diagram
This will also find the maximum distance to consider in order to fill a certain amount of the histogram. The amount is set by coverage

*TODO: Check how the distance function works*

*TODO: Make first color (background) black*

In [16]:
coverage = 0.9
phasediagram = registration.phase_diagram(H,xedges,yedges,opts,coverage=coverage)
nphases=phasediagram.max()

#plot the phasediagram
cmap = 'Set3' #vill bara ha x färger
Y,X = np.meshgrid(yedges,xedges,indexing='ij') #coordinates in dual histogram
fig=plt.figure()
m=plt.pcolormesh(X,Y,phasediagram,cmap=cmap)
plt.xlabel('X-ray intensity')
plt.ylabel('Neutron intensity')
cb=fig.colorbar(m,ticks=range(nphases+1))
m.set_clim(0.5, nphases + 0.5)


Finding full coverage: sigma=2, coverage=0.35541653690479946
Finding full coverage: sigma=3, coverage=0.6248044207411518
Finding full coverage: sigma=4, coverage=0.8168229665057433
Finding full coverage: sigma=5, coverage=0.8917302939951289
Finding full coverage: sigma=6, coverage=0.9067121990039733


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.figure()
plt.imshow(phasediagram,origin='lower',cmap=cmap)

## Map from the phase diagram to the volume
This create a segmented volume. It is possible to label just one slice by giving slicenr. slicenr=None will label the full volume.

*TODO Check if this is terribly slow for a real volume*

In [25]:
#%%time
slicenr = 750
labels = registration.map_to_volume(x_volume,n_volume,xedges,yedges,phasediagram,slicenr=slicenr)
labels = labels.astype(np.uint8)
print('Assigned voxels of each phase:')
bc = np.bincount(labels.ravel())
for i,b in enumerate(bc):
    print('Phase {:d}: {:d} voxels'.format(i,b))

Labelling slice 750 of 1220
Assigned voxels of each phase:
Phase 0: 238642 voxels
Phase 1: 2914 voxels
Phase 2: 29628 voxels
Phase 3: 71016 voxels


In [26]:
if not slicenr:
    z,x,y = labels.shape[0]//2, labels.shape[1]//2,labels.shape[2]//2
    fig,axes=plt.subplots(1,3,figsize=(12,6))
    ax = axes.ravel()
    fig.suptitle('Labelled slices')
    m=ax[0].imshow(labels[z],cmap=cmap,vmin=1)
    ax[1].imshow(labels[:,x,:],cmap=cmap,vmin=1)
    ax[2].imshow(labels[:,:,y],cmap=cmap,vmin=1)
    cb=fig.colorbar(m,ticks=range(nphases+1))
    m.set_clim(0.5, nphases + 0.5)

    fig,axes=plt.subplots(1,3,figsize=(12,6))
    ax = axes.ravel()
    fig.suptitle('X-ray data')
    ax[0].imshow(x_volume[z],cmap='Greys_r')
    ax[1].imshow(x_volume_def[:,x,:],cmap='Greys_r')
    ax[2].imshow(x_volume_def[:,:,y],cmap='Greys_r')

    fig,axes=plt.subplots(1,3,figsize=(12,6))
    ax = axes.ravel()
    fig.suptitle('Neutron data')
    ax[0].imshow(n_volume[z],cmap='Greys_r')
    ax[1].imshow(n_volume[:,x,:],cmap='Greys_r')
    ax[2].imshow(n_volume[:,:,y],cmap='Greys_r')
else:
    fig,axes=plt.subplots(1,3,figsize=(12,6),sharex=True,sharey=True)
    ax = axes.ravel()
    ax[0].set_title('Labelled slices')
    m=ax[0].imshow(labels,cmap=cmap,vmin=1)
    cb=fig.colorbar(m,ticks=range(nphases+1))
    m.set_clim(0.5, nphases + 0.5)
    ax[1].set_title('X-ray data')
    ax[1].imshow(x_volume[slicenr],cmap='Greys_r')
    ax[2].set_title('Neutron data')
    ax[2].imshow(n_volume[slicenr],cmap='Greys_r')
 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.figure()
plt.imshow(labels==1)